### Prerequisites

In [21]:
import pandas as pd
import re

#### Data Loading

In [ ]:
# Load data into a pandas df
df = pd.read_csv('../data/movies.csv')

# Verify data is loaded correctly
## Check rows + cols exist
num_rows, num_cols = df.shape
print(f"Row Count: {num_rows}")
print(f"Column Count: {num_cols}")

## Data Specs
print(df.info())

Row Count: 15509
Column Count: 10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB
None


#### Data Preprocessing

#### Helper Function

In [23]:
# Print count + percentage
def print_missing(df, col_name, total_rows):
    missing = df[col_name].isna().sum()
    
    # If the column is a string/object type, also count empty strings
    if df[col_name].dtype == 'object':
        missing += (df[col_name].astype(str).str.strip() == '').sum()
    
    print(f"Missing {col_name} values: {missing} ({missing / total_rows:.2%})")

# Remove 'min' and convert to int using regex
def clean_duration(value):
    if pd.isna(value):
        return None
    
    # RegEx to extract the first group of digits
    match = re.search(r'\d+', str(value))
    
    if match:
        return int(match.group())
    else:
        return None
    
# Convert to string and extract a 4-digit number using regex
def clean_year(value):
    if pd.isna(value):
        return None
    
    # RegEx to extract year
    match = re.search(r'\d{4}', str(value))
    
    if match:
        return int(match.group())
    else:
        return None
    
# Clean votes column by replacing commas
def clean_votes(value):
    if pd.isna(value):
        return None
    
    # Remove commas + convert to int
    value = str(value).replace(',', '').strip()
    
    # Return as int if it's a valid number
    return int(value) if value.isdigit() else None


##### Analyze which columns require attention


In [24]:
total_rows = len(df)

print_missing(df, 'Name', total_rows)
print_missing(df, 'Year', total_rows)
print_missing(df, 'Duration', total_rows)
print_missing(df, 'Genre', total_rows)
print_missing(df, 'Rating', total_rows)
print_missing(df, 'Votes', total_rows)
print_missing(df, 'Director', total_rows)
print_missing(df, 'Actor 1', total_rows)
print_missing(df, 'Actor 2', total_rows)
print_missing(df, 'Actor 3', total_rows)

print(f"Total rows: {total_rows}")

Missing Name values: 1 (0.01%)
Missing Year values: 528 (3.40%)
Missing Duration values: 8269 (53.32%)
Missing Genre values: 1877 (12.10%)
Missing Rating values: 7590 (48.94%)
Missing Votes values: 7589 (48.93%)
Missing Director values: 525 (3.39%)
Missing Actor 1 values: 1617 (10.43%)
Missing Actor 2 values: 2384 (15.37%)
Missing Actor 3 values: 3144 (20.27%)
Total rows: 15509


##### Handling Errors + Exceptions

In [25]:
print(df.head())

# Drop rows with missing Year
df = df.dropna(subset=['Year'])

# Remove '-' and convert to integer
df['Year'] = df['Year'].apply(clean_year)

# Clean and convert to int format
df['Duration'] = df['Duration'].apply(clean_duration)
df['Duration_missing'] = df['Duration'].isna()

# Convert votes to usable ints
df['Votes'] = df['Votes'].apply(clean_votes)

# Verify results are as expected
df

                                 Name    Year Duration            Genre  \
0                                         NaN      NaN            Drama   
1  #Gadhvi (He thought he was Gandhi)  (2019)  109 min            Drama   
2                         #Homecoming  (2021)   90 min   Drama, Musical   
3                             #Yaaram  (2019)  110 min  Comedy, Romance   
4                   ...And Once Again  (2010)  105 min            Drama   

   Rating Votes            Director       Actor 1             Actor 2  \
0     NaN   NaN       J.S. Randhawa      Manmauji              Birbal   
1     7.0     8       Gaurav Bakshi  Rasika Dugal      Vivek Ghamande   
2     NaN   NaN  Soumyajit Majumdar  Sayani Gupta   Plabita Borthakur   
3     4.4    35          Ovais Khan       Prateik          Ishita Raj   
4     NaN   NaN        Amol Palekar  Rajat Kapoor  Rituparna Sengupta   

           Actor 3  
0  Rajendra Bhatia  
1    Arvind Jangid  
2       Roy Angana  
3  Siddhant Kapoor  
4    

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Duration_missing
1,#Gadhvi (He thought he was Gandhi),2019,109.0,Drama,7.0,8.0,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,False
2,#Homecoming,2021,90.0,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,False
3,#Yaaram,2019,110.0,"Comedy, Romance",4.4,35.0,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,False
4,...And Once Again,2010,105.0,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali,False
5,...Aur Pyaar Ho Gaya,1997,147.0,"Comedy, Drama, Musical",4.7,827.0,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,False
...,...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,1988,NaN,Action,4.6,11.0,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand,True
15505,Zulmi,1999,129.0,"Action, Drama",4.5,655.0,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani,False
15506,Zulmi Raj,2005,NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN,True
15507,Zulmi Shikari,1988,NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN,True


##### Save cleaned table

In [26]:
# Save processed table
df.to_csv('../data/movies_cleaned.csv', index=False)